# This notebook has some utilities to manage the already deployed cluster. 
- uninstall DYNAMOS
- trigger VFL
- ... 

## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
%%time
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

User: koufalex@gmail.com bastion key is valid!
Configuration is valid


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,koufalex_0000215529
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


CPU times: user 1.92 s, sys: 217 ms, total: 2.14 s
Wall time: 4.16 s


In [2]:
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();
print(nodes)


[<fabrictestbed_extensions.fablib.node.Node object at 0x7e23d96170d0>, <fabrictestbed_extensions.fablib.node.Node object at 0x7e23daebdbd0>, <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc788e10>, <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc769a90>, <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc74f910>, <fabrictestbed_extensions.fablib.node.Node object at 0x7e241017c390>]


In [3]:
# Print the necessary information
try:
    # Get slice nodes
    for node in slice.get_nodes():
        print(f"Node: {node.get_name()}")
        # Get the original SSH command
        original_ssh_command = node.get_ssh_command()
        # Print SSH commands to get into the nodes
        print(f"  SSH Command from FABRIC: {original_ssh_command}")
        # Replace the file paths in the SSH command
        updated_ssh_command = original_ssh_command.replace(
            "/home/fabric/work/fabric_config/slice_key", "~/.ssh_fabric/slice_key"
        ).replace(
            "/home/fabric/work/fabric_config/ssh_config", "ssh_config"
        )
        # Print the updated SSH command
        print(f"  SSH Command locally (ensuring it is saved according to below steps): {updated_ssh_command}")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node: control
  SSH Command from FABRIC: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe19:b747
  SSH Command locally (ensuring it is saved according to below steps): ssh -i ~/.ssh_fabric/slice_key -F ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe19:b747
Node: dynamos
  SSH Command from FABRIC: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fef2:4910
  SSH Command locally (ensuring it is saved according to below steps): ssh -i ~/.ssh_fabric/slice_key -F ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fef2:4910
Node: server
  SSH Command from FABRIC: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe63:3953
  SSH Command locally (ensuring it is saved according to below steps): ssh -i ~/.ssh_fabric/slice_key -F ssh_config ubuntu@2001:400

In [4]:
def get_ip(node):
    interface = node.get_interface(network_name=f"Network-{node.get_site()}")
    return interface.get_ip_addr()

nodes_dict= dict()

for node in nodes[:]:
    ip = get_ip(node)
    name = node.get_name()
    nodes_dict[name] = {"ip": ip, "node": node}
    print(f"{name}: {ip}")

print(nodes_dict)

control: 10.137.8.2
dynamos: 10.137.8.3
server: 10.137.8.4
clientone: 10.137.8.5
clienttwo: 10.137.8.6
clientthree: 10.137.8.7
{'control': {'ip': '10.137.8.2', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e23d96170d0>}, 'dynamos': {'ip': '10.137.8.3', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e23daebdbd0>}, 'server': {'ip': '10.137.8.4', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc788e10>}, 'clientone': {'ip': '10.137.8.5', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc769a90>}, 'clienttwo': {'ip': '10.137.8.6', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e23cc74f910>}, 'clientthree': {'ip': '10.137.8.7', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7e241017c390>}}


In [ ]:
# This is for resetting the kubespray cluster. 
# Use this if you are troubleshooting your Kubernetes cluster
# and you want to redeploy fresh.

# nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/reset_kubespray.sh", remote_file_path="reset.sh");
# nodes_dict['control']['node'].execute(f"chmod +x reset.sh && ./reset.sh");

In [ ]:
# Add the relevant etcd data to the dynamos node (update etcd agreements etc)

# help(nodes_dict['dynamos']['node'])
# upload etcd files from filesystem instead of reading them from github
# nodes_dict['dynamos']['node'].upload_directory(local_directory_path="../configuration/etcd_launch_files", remote_directory_path="./")
# nodes_dict['dynamos']['node'].execute("ls etcd_launch_files")

# nodes_dict['dynamos']['node'].upload_file(local_file_path="node_scripts/define_etcd_data_local.sh", remote_file_path="define_etcd_data_local.sh");
# nodes_dict['dynamos']['node'].execute(f"chmod +x define_etcd_data_local.sh && ./define_etcd_data_local.sh");

In [8]:
%%time
# Run the trigger script
# note: could make it so that ignores the output > null 
# nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/trigger_VFL.sh", remote_file_path="trigger_VFL.sh")

nodes_dict['control']['node'].execute("bash trigger_VFL.sh")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   409    0     0  100   409      0      9  0:00:45  0:00:42  0:00:03     0
CPU times: user 125 ms, sys: 40.7 ms, total: 166 ms
Wall time: 44 s


('',
 '  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100   409    0     0  100   409      0    408  0:00:01  0:00:01 --:--:--   408\r100   409    0     0  100   409      0    204  0:00:02  0:00:02 --:--:--   204\r100   409    0     0  100   409      0    136  0:00:03  0:00:03 --:--:--   136\r100   409    0     0  100   409      0    102  0:00:04  0:00:04 --:--:--   102\r100   409    0     0  100   409      0     81  0:00:05  0:00:05 --:--:--    81\r100   409    0     0  100   409      0     68  0:00:06  0:00:06 --:--:--     0\r100   409    0     0  100   409      0     58  0:00:07  0:00:07 --:--:--     0\r100   409    0     0  100   409      0     51  0:00:08  0:00:08 --:--:--     0\r100   409    0     0  100   409      0     45  0:00:09  0:00:09 --:--:--     0\r100   409    0     0  100   409

In [14]:
%%time

nodes_dict['control']['node'].upload_file(local_file_path="../scripts/retrieve_logs.sh", remote_file_path="retrieve_logs.sh")
nodes_dict['control']['node'].execute(f"bash retrieve_logs.sh")

1.75941333664469e+09 WARN	/app/pkg/lib/go-grpc.go:38	could not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"
1.7594133366447582e+09 DEBUG	/app/pkg/lib/go-grpc.go:44	Sleep 1 second
1.759413337645562e+09 WARN	/app/pkg/lib/go-grpc.go:38	could not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"
1.7594133376456206e+09 DEBUG	/app/pkg/lib/go-grpc.go:44	Sleep 1 second
1.759413338646503e+09 WARN	/app/pkg/lib/go-grpc.go:38	could not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"
1.7594133386465685e+09 DEBUG	/app/pkg/lib/go-grpc.go:44	Sleep 1 second
1.759413339649266e+09 DEBUG	/app/pkg/lib/go-grpc.go:51	returning conn GetGrpcConnection
1.7594133396496134e+09 DEBUG	/app/pkg

('1.75941333664469e+09 \x1bWARN\x1b\t/app/pkg/lib/go-grpc.go:38\tcould not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"\n1.7594133366447582e+09 \x1bDEBUG\x1b\t/app/pkg/lib/go-grpc.go:44\tSleep 1 second\n1.759413337645562e+09 \x1bWARN\x1b\t/app/pkg/lib/go-grpc.go:38\tcould not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"\n1.7594133376456206e+09 \x1bDEBUG\x1b\t/app/pkg/lib/go-grpc.go:44\tSleep 1 second\n1.759413338646503e+09 \x1bWARN\x1b\t/app/pkg/lib/go-grpc.go:38\tcould not check: rpc error: code = Unavailable desc = connection error: desc = "transport: Error while dialing: dial tcp 127.0.0.1:50051: connect: connection refused"\n1.7594133386465685e+09 \x1bDEBUG\x1b\t/app/pkg/lib/go-grpc.go:44\tSleep 1 second\n1.759413339649266e+09 \x1bDEBUG\x1b\t/app/pkg/lib/go-grpc

In [11]:
%%time
# TODO: read the metrics based on the other script 

nodes_dict['control']['node'].upload_file(local_file_path="../scripts/retrieve_data.sh", remote_file_path="retrieve_data.sh")
nodes_dict['control']['node'].execute(f"bash retrieve_data.sh {2}")


40.49493813273341
40.607424071991
40.49493813273341
40.49493813273341
40.607424071991
40.83239595050618
41.05736782902137
41.05736782902137
41.16985376827896
41.39482564679415
41.16985376827896
41.28233970753656
41.732283464566926
41.732283464566926
41.61979752530934
41.84476940382452
41.84476940382452
41.95725534308212
41.95725534308212
CPU times: user 52.2 ms, sys: 14.9 ms, total: 67.2 ms
Wall time: 3.63 s


('40.49493813273341\n40.607424071991\n40.49493813273341\n40.49493813273341\n40.607424071991\n40.83239595050618\n41.05736782902137\n41.05736782902137\n41.16985376827896\n41.39482564679415\n41.16985376827896\n41.28233970753656\n41.732283464566926\n41.732283464566926\n41.61979752530934\n41.84476940382452\n41.84476940382452\n41.95725534308212\n41.95725534308212\n',
 '')

In [12]:
for node in nodes:
    print(node)
    ssh_command = node.get_ssh_command().replace(
        "-i /home/fabric/work/fabric_config/slice_key", "-i ~/.ssh/keys/FABRIC-slice_key"
    ).replace(
        "-F /home/fabric/work/fabric_config/ssh_config ", ""
    )
    
    print(ssh_command);

-----------------  --------------------------------------------------------------------------------------------------------------------------------------------
ID                 7e6ecd76-c04e-4227-9f31-3f4201a053ba
Name               control
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_24
Image Type         qcow2
Host               losa-w3.fabric-testbed.net
Site               LOSA
Management IP      2001:400:a100:3070:f816:3eff:fe84:397d
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe84:397d
-----------------  --------------------------------------------------------------------------------------------------------------------------------------------
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe84:397d
-----------------  -----------------------------------------

In [7]:
# Do we need to do it everytime?? I don;t think so but I should test 

slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

for node in nodes:
    ssh_command = node.get_ssh_command().replace(
        "-i /home/fabric/work/fabric_config/slice_key", "-i ~/.ssh/keys/FABRIC-slice_key"
    ).replace(
        "-F /home/fabric/work/fabric_config/ssh_config ", ""
    )
    
    print(ssh_command);

Uploading the node setup...


[<SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572167 mtime=1758572168 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572165 mtime=1758572165 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1758572167 mtime=1758572168 ]>]

In [23]:

# Configure DYNAMOS for the FABRIC nodes
# agents_string = ",".join(agents)
# thirdparties_string = ",".join(thirdparties)


# nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/configure_dynamos.sh", remote_file_path="configure_dynamos.sh");
# nodes_dict['control']['node'].execute(f"chmod +x configure_dynamos.sh && ./configure_dynamos.sh {agents_string} {thirdparties_string}");

Adding agents...
- agent 'server'
- agent 'clientone'
- agent 'clienttwo'
- agent 'clientthree'

Adding third parties...


In [24]:
# Optionally override the installation scripts 
# nodes_dict['control']['node'].upload_file(local_file_path="../configuration/dynamos-configuration.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/dynamos-configuration.sh")
# nodes_dict['control']['node'].upload_file(local_file_path="../configuration/fill-rabbit-pvc.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/fill-rabbit-pvc.sh");


In [16]:
# install DYNAMOS
nodes_dict['control']['node'].execute(f"~/DYNAMOS/configuration/dynamos-configuration.sh")

DYNAMOS configuration v0.1.2
Setting up paths...
definitions_example.json copied over definitions.json to ensure a clean file
Generating RabbitMQ password...
Replacing tokens...
Installing namespaces...
Release "namespaces" does not exist. Installing it now.
NAME: namespaces
LAST DEPLOYED: Thu Oct  2 14:41:54 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
Preparing PVC
fill-pvc v0.1.2
pod/temp-pod created
pod/temp-pod-orch created
Waiting for temp-pod to be Running...
pod/temp-pod condition met
pod/temp-pod-orch condition met
E1002 14:41:58.355966 2709862 websocket.go:296] Unknown stream id 1, discarding message
pod "temp-pod" deleted
pod "temp-pod-orch" deleted
Installing Prometheus...
"prometheus-community" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "prometheus-community" chart repository
Update Complete. ⎈Happy Helming!⎈
Release "prometheus" doe

('DYNAMOS configuration v0.1.2\nSetting up paths...\ndefinitions_example.json copied over definitions.json to ensure a clean file\nGenerating RabbitMQ password...\nReplacing tokens...\nInstalling namespaces...\nRelease "namespaces" does not exist. Installing it now.\nNAME: namespaces\nLAST DEPLOYED: Thu Oct  2 14:41:54 2025\nNAMESPACE: default\nSTATUS: deployed\nREVISION: 1\nTEST SUITE: None\nPreparing PVC\nfill-pvc v0.1.2\npod/temp-pod created\npod/temp-pod-orch created\nWaiting for temp-pod to be Running...\npod/temp-pod condition met\npod/temp-pod-orch condition met\npod "temp-pod" deleted\npod "temp-pod-orch" deleted\nInstalling Prometheus...\n"prometheus-community" already exists with the same configuration, skipping\nHang tight while we grab the latest from your chart repositories...\n...Successfully got an update from the "prometheus-community" chart repository\nUpdate Complete. ⎈Happy Helming!⎈\nRelease "prometheus" does not exist. Installing it now.\nNAME: prometheus\nLAST DEP

In [15]:
# Optional to clean up: uninstall DYNAMOS

command = "helm uninstall agents api-gateway core orchestrator namespaces prometheus thirdparties"
# nodes_dict['control']['node'].execute(command)

release "agents" uninstalled
release "api-gateway" uninstalled
release "core" uninstalled
release "orchestrator" uninstalled
These resources were kept due to the resource policy:
[Namespace] core
[Namespace] orchestrator
[Namespace] clienttwo
[Namespace] clientthree
[Namespace] uva
[Namespace] vu
[Namespace] surf
[Namespace] ingress
[Namespace] api-gateway
[Namespace] alpha
[Namespace] server
[Namespace] clientone

release "namespaces" uninstalled
release "prometheus" uninstalled
release "thirdparties" uninstalled


('release "agents" uninstalled\nrelease "api-gateway" uninstalled\nrelease "core" uninstalled\nrelease "orchestrator" uninstalled\nThese resources were kept due to the resource policy:\n[Namespace] core\n[Namespace] orchestrator\n[Namespace] clienttwo\n[Namespace] clientthree\n[Namespace] uva\n[Namespace] vu\n[Namespace] surf\n[Namespace] ingress\n[Namespace] api-gateway\n[Namespace] alpha\n[Namespace] server\n[Namespace] clientone\n\nrelease "namespaces" uninstalled\nrelease "prometheus" uninstalled\nrelease "thirdparties" uninstalled\n',
 '')

In [14]:
# Uninstall client and server namespaces (sometimes is needed)
command = "kubectl delete namespace clientone clienttwo clientthree server"
# nodes_dict['control']['node'].execute(command)

namespace "clientone" deleted
namespace "clienttwo" deleted
namespace "clientthree" deleted
namespace "server" deleted


('namespace "clientone" deleted\nnamespace "clienttwo" deleted\nnamespace "clientthree" deleted\nnamespace "server" deleted\n',
 '')

In [ ]:
# Optional delete etcd PVCs 
# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")

# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-0 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-1 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-2 -n core")


# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")